
## Capstone Project
by *Mathias Jørgensen*


## Introduction:

I am working for a high-end restaurant chain and am trying to an area to open a Michellin-type restaurant in.
My stakeholders, the investors in EATWELL Corp, are very keen on finding the perfect location to open this new restaurant.

Having conversed with several of my teams members and brainstormed, we have defined the spot for us would be:
- An area where the housing prices are above the average in London.
- A minimum amount of restaurants already.

We need to present the perfect location to our stakeholders, the investors.


## Data

To get information on all property transactions in London, we have found the data which HM Land Registry publishes on their website. (http://landregistry.data.gov.uk/). All of this information is publicly available due to the government wanting to promote economic growth and data transparency. A total of three datasets are available, with the first one being our main priority in this report:

- Price paid data updated monthly, data available from 1995.
- Transaction data updated monthly, data available from December 2011.
- UK House Price Index downloads updated monthly, data available from January 1995.

The datasets come in comma-separated value (csv) and linked data formats, with Price Paid Data also available as a text file. The dataset we will use for this report will be download from the following link for the 2019 version: http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv

In the Price Paid Data, we have a broad selection of information, including the the following information:
- Postcode 
- PAON Primary Addressable Object Name (Typically the house number or name)
- SAON Secondary Addressable Object Name (If there is a sub-building, for example, the building is divided into flats, there will be a SAON) 
- Street
- Locality 
- Town/City 
- District 
- County

Furthermore, to explorer the restaurants in the areas we will access the FourSquare API and combine this information with the above dataset to find an area which is both in the high end of housing prices and with other well-reviewed restaurants near in order for us to identify the area as a good fit for a new restaurant.


# Methodology 


In [3]:
#IN THIS SECTION OF THE CODE, WE IMPORT ALL THE LIBRARIES NEEDED FOR THE REPORT

import os # Operating System
import sys
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Data preparation and preprocessing

Here we prepare the dataset by doing the following steps:

- Name the columns needed
- Reformat the date 
- Sort by date of sale
- Choose only data for the city of London
- Make a list of street names in London
- Calculate the average price of the property per street
- Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
- Join the data to find the coordinates



In [13]:
#Import the 2019 data from the UK Land Registry (http://landregistry.data.gov.uk/)
#importdata = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv")

PriceData = importdata

PriceData.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

#We only want to look at transactions in the last five years, so we delete all transactions before 2016
PriceData['Date_Transfer'] = pd.to_datetime(PriceData['Date_Transfer'])
start_date = '01-01-2016'
end_date = '31-12-2020'
mask = (PriceData['Date_Transfer'] > start_date) & (PriceData['Date_Transfer'] <= end_date)
PriceData = PriceData.loc[mask]
PriceData = PriceData.sort_values(by='Date_Transfer', ascending=False, na_position='first')
PriceData

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
167663,{98C75471-DBF0-72E9-E053-6B04A8C042F0},100000,2019-11-28,NN1 4JS,T,N,F,10,NaN,DERBY ROAD,ABINGTON,NORTHAMPTON,NORTHAMPTON,NORTHAMPTONSHIRE,A,A
187466,{98C75472-296B-72E9-E053-6B04A8C042F0},210000,2019-11-28,NN9 5JB,S,N,F,20,NaN,TINGDENE ROAD,FINEDON,WELLINGBOROUGH,WELLINGBOROUGH,NORTHAMPTONSHIRE,A,A
743689,{98C75471-AD6F-72E9-E053-6B04A8C042F0},299000,2019-11-28,BH18 8HU,D,N,F,75,NaN,KEIGHLEY AVENUE,NaN,BROADSTONE,"BOURNEMOUTH, CHRISTCHURCH AND POOLE","BOURNEMOUTH, CHRISTCHURCH AND POOLE",A,A
114116,{98C75471-959C-72E9-E053-6B04A8C042F0},145000,2019-11-28,PL18 9NQ,D,N,F,THORNLEIGH,NaN,NaN,MIDDLE DIMSON,GUNNISLAKE,CORNWALL,CORNWALL,A,A
212676,{98C75472-BCCC-72E9-E053-6B04A8C042F0},102500,2019-11-28,EC4Y 8AD,O,N,F,4 - 7,ROOF AND AIR SPACE,LOMBARD LANE,NaN,LONDON,CITY OF LONDON,GREATER LONDON,B,A
646941,{98C75472-A7AB-72E9-E053-6B04A8C042F0},35500,2019-11-27,LA14 1DR,T,N,F,34,NaN,HALL STREET,NaN,BARROW-IN-FURNESS,BARROW-IN-FURNESS,CUMBRIA,B,A
722110,{98C75472-0003-72E9-E053-6B04A8C042F0},175000,2019-11-27,LN4 4DQ,D,N,F,1,NaN,FERRY LANE,NORTH KYME,LINCOLN,NORTH KESTEVEN,LINCOLNSHIRE,A,A
222599,{98C75472-C633-72E9-E053-6B04A8C042F0},61000,2019-11-27,S26 5LX,T,N,F,63,NaN,EAST TERRACE,WALES BAR,SHEFFIELD,ROTHERHAM,SOUTH YORKSHIRE,B,A
194618,{98C75472-1F5D-72E9-E053-6B04A8C042F0},3750000,2019-11-27,SW1W 8JL,T,N,F,65,NaN,CHESTER ROW,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A
114247,{98C75471-97D1-72E9-E053-6B04A8C042F0},285000,2019-11-27,LA8 8NF,S,N,F,20,NaN,GREENGATE,LEVENS,KENDAL,SOUTH LAKELAND,CUMBRIA,A,A


In [51]:
#Create a new dataframe which only has data for London, which is the city we are interested in
PriceData_London = PriceData.query("Town_City == 'LONDON'")
PriceData_London



,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
212676,{98C75472-BCCC-72E9-E053-6B04A8C042F0},102500,2019-11-28,EC4Y 8AD,O,N,F,4 - 7,ROOF AND AIR SPACE,LOMBARD LANE,NaN,LONDON,CITY OF LONDON,GREATER LONDON,B,A
194618,{98C75472-1F5D-72E9-E053-6B04A8C042F0},3750000,2019-11-27,SW1W 8JL,T,N,F,65,NaN,CHESTER ROW,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,A,A
182394,{98C75472-1CA0-72E9-E053-6B04A8C042F0},677000,2019-11-26,E1 4BH,T,N,F,4,NaN,MILE END PLACE,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A
199192,{98C75472-BBCC-72E9-E053-6B04A8C042F0},32500,2019-11-25,N17 0DT,O,N,F,79,NaN,BRANTWOOD ROAD,NaN,LONDON,HARINGEY,GREATER LONDON,B,A
215626,{98C75472-BD01-72E9-E053-6B04A8C042F0},1141122,2019-11-25,W1W 7LX,F,N,L,89,FLAT 1,GREAT PORTLAND STREET,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A
194831,{98C75471-6EE6-72E9-E053-6B04A8C042F0},385000,2019-11-25,W13 8SB,F,N,L,156B,NaN,UXBRIDGE ROAD,NaN,LONDON,EALING,GREATER LONDON,A,A
632237,{98C75472-A06A-72E9-E053-6B04A8C042F0},34000,2019-11-25,E17 9RN,O,N,F,53,NaN,CHURCH LANE,WALTHAMSTOW,LONDON,WALTHAM FOREST,GREATER LONDON,B,A
111200,{98C75472-434D-72E9-E053-6B04A8C042F0},585000,2019-11-25,SW11 5QU,T,N,F,56,NaN,DUNSTON ROAD,NaN,LONDON,WANDSWORTH,GREATER LONDON,A,A
641009,{98C75472-ADD8-72E9-E053-6B04A8C042F0},5000,2019-11-25,N4 4AP,O,N,L,11,NaN,CROUCH HILL,NaN,LONDON,ISLINGTON,GREATER LONDON,B,A
211281,{98C75472-1C52-72E9-E053-6B04A8C042F0},1270000,2019-11-25,NW8 7PX,F,N,L,"LONDON HOUSE, 7 - 9",FLAT 50,AVENUE ROAD,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A


In [122]:
#Make a list called "Streets" of the streets in London
Streets_London = PriceData_London['Street'].unique().tolist()

PriceData_London_ByPrice = PriceData_London.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
PriceData_London_ByPrice.columns = ['Street', 'Avg_Price']

# Only take the top n most expensive neighborhoods
n = 10
PriceData_London_ByPrice = PriceData_London_ByPrice.sort_values('Avg_Price', ascending = False).head(n)

# Display the dataframe
PriceData_London_ByPrice


,Street,Avg_Price
5203,GRESHAM STREET,411500000.0
4967,GODLIMAN STREET,302401851.0
8986,PENNANT MEWS,194800000.0
7845,MICHAEL ROAD,189089600.0
193,ALDGATE HIGH STREET,183000000.0
5433,HANOVER SQUARE,165500000.0
2100,CARTER LANE,159660000.0
6223,IMPERIAL ROAD,157940000.0
2879,COOPERS ROW,152706477.6
12407,WHITEHALL,107400000.0


In [109]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans



In [110]:
for index, item in PriceData_London_ByPrice.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 5203
item: Street       GRESHAM STREET
Avg_Price         4.115e+08
Name: 5203, dtype: object
item.Street only: GRESHAM STREET
index: 4967
item: Street       GODLIMAN STREET
Avg_Price        3.02402e+08
Name: 4967, dtype: object
item.Street only: GODLIMAN STREET
index: 8986
item: Street       PENNANT MEWS
Avg_Price       1.948e+08
Name: 8986, dtype: object
item.Street only: PENNANT MEWS
index: 7845
item: Street       MICHAEL ROAD
Avg_Price      1.8909e+08
Name: 7845, dtype: object
item.Street only: MICHAEL ROAD
index: 193
item: Street       ALDGATE HIGH STREET
Avg_Price               1.83e+08
Name: 193, dtype: object
item.Street only: ALDGATE HIGH STREET
index: 5433
item: Street       HANOVER SQUARE
Avg_Price         1.655e+08
Name: 5433, dtype: object
item.Street only: HANOVER SQUARE
index: 2100
item: Street       CARTER LANE
Avg_Price     1.5966e+08
Name: 2100, dtype: object
item.Street only: CARTER LANE
index: 6223
item: Street       IMPERIAL ROAD
Avg_Price       1.5794e+08
Na

In [114]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

# Print a row with more detailed information about the streets for each street
for street in PriceData_London_ByPrice['Street']:
    print(geolocator.geocode(street))


Gresham Street, Canoga Park, Los Angeles, Los Angeles County, California, 91306, United States of America
Godliman Street, Temple, City of London, Greater London, England, EC4V 5BD, United Kingdom
Pennant Mews, Earl's Court, Royal Borough of Kensington and Chelsea, London, Greater London, England, W8 5JJ, United Kingdom
Michael Road, West Deer Township, Allegheny County, Pennsylvania, 15076, United States of America
Aldgate High Street, City of London, Greater London, England, EC3N 1BD, United Kingdom
Hanover Square, St. James's, Mayfair, City of Westminster, London, Greater London, England, United Kingdom
Carter Lane, Brookland, Folkestone and Hythe, Kent, South East, England, TN29 9RL, United Kingdom
Imperial Road, Haringey Heartlands, London Borough of Haringey, London, Greater London, England, N22 8DE, United Kingdom
Coopers Row, City of London, Greater London, England, EC3N 3AE, United Kingdom
Whitehall, London, Greater London, England, SW1A 2NH, United Kingdom


In [115]:
#In the above outputs it's apparent that the geolocator thinks some of the streets are in the US and not London.
#We can fix this by adding ", London" to the search

for street in PriceData_London_ByPrice['Street']:
    print(geolocator.geocode(street + ", London"))

Gresham Street, Temple, City of London, Greater London, England, EC2V 7BX, United Kingdom
Godliman Street, Temple, City of London, Greater London, England, EC4V 5BD, United Kingdom
Pennant Mews, Earl's Court, Royal Borough of Kensington and Chelsea, London, Greater London, England, W8 5JJ, United Kingdom
Michael Road, Whipps Cross, Leytonstone, London Borough of Waltham Forest, London, Greater London, England, E11 3DY, United Kingdom
Aldgate High Street, City of London, Greater London, England, EC3N 1BD, United Kingdom
Hanover Square, St. James's, Mayfair, City of Westminster, London, Greater London, England, United Kingdom
Carter Lane, Temple, City of London, Greater London, England, EC4V 5AE, United Kingdom
Imperial Road, Haringey Heartlands, London Borough of Haringey, London, Greater London, England, N22 8QQ, United Kingdom
Coopers Row, City of London, Greater London, England, EC3N 3AE, United Kingdom
Whitehall, London, Greater London, England, SW1A 2NH, United Kingdom


In [123]:
#We can fix this in the dataframe by adding this as well

PriceData_London_ByPriceFixed = PriceData_London_ByPrice
PriceData_London_ByPriceFixed['Street'] = PriceData_London_ByPrice['Street'].astype(str) + ", LONDON"
PriceData_London_ByPriceFixed


,Street,Avg_Price
5203,"GRESHAM STREET, LONDON",411500000.0
4967,"GODLIMAN STREET, LONDON",302401851.0
8986,"PENNANT MEWS, LONDON",194800000.0
7845,"MICHAEL ROAD, LONDON",189089600.0
193,"ALDGATE HIGH STREET, LONDON",183000000.0
5433,"HANOVER SQUARE, LONDON",165500000.0
2100,"CARTER LANE, LONDON",159660000.0
6223,"IMPERIAL ROAD, LONDON",157940000.0
2879,"COOPERS ROW, LONDON",152706477.6
12407,"WHITEHALL, LONDON",107400000.0


In [131]:
PriceData_London_ByPriceFixed['street_coordinates'] = PriceData_London_ByPriceFixed['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
PriceData_London_ByPriceFixed

,Street,Avg_Price,Latitude,Longitude,street_coordinates
5203,"GRESHAM STREET, LONDON",411500000.0,51.516232,-0.096448,"(51.5162317, -0.0964481)"
4967,"GODLIMAN STREET, LONDON",302401851.0,51.512526,-0.099089,"(51.5125257, -0.0990892)"
8986,"PENNANT MEWS, LONDON",194800000.0,51.495338,-0.191601,"(51.4953378, -0.1916006)"
7845,"MICHAEL ROAD, LONDON",189089600.0,51.566179,0.011433,"(51.5661793, 0.0114331)"
193,"ALDGATE HIGH STREET, LONDON",183000000.0,51.513980,-0.075425,"(51.5139801, -0.0754254)"
5433,"HANOVER SQUARE, LONDON",165500000.0,51.513908,-0.143824,"(51.51390775, -0.14382390827702274)"
2100,"CARTER LANE, LONDON",159660000.0,51.512973,-0.100203,"(51.5129732, -0.1002027)"
6223,"IMPERIAL ROAD, LONDON",157940000.0,51.602725,-0.122014,"(51.6027248, -0.1220144)"
2879,"COOPERS ROW, LONDON",152706477.6,51.511590,-0.077344,"(51.5115896, -0.077344)"
12407,"WHITEHALL, LONDON",107400000.0,51.502328,-0.126083,"(51.5023278, -0.1260826)"


In [132]:
PriceData_London_ByPriceFixed[['Latitude', 'Longitude']] = PriceData_London_ByPriceFixed['street_coordinates'].apply(pd.Series)

PriceData_London_ByPriceFixed

,Street,Avg_Price,Latitude,Longitude,street_coordinates
5203,"GRESHAM STREET, LONDON",411500000.0,51.516232,-0.096448,"(51.5162317, -0.0964481)"
4967,"GODLIMAN STREET, LONDON",302401851.0,51.512526,-0.099089,"(51.5125257, -0.0990892)"
8986,"PENNANT MEWS, LONDON",194800000.0,51.495338,-0.191601,"(51.4953378, -0.1916006)"
7845,"MICHAEL ROAD, LONDON",189089600.0,51.566179,0.011433,"(51.5661793, 0.0114331)"
193,"ALDGATE HIGH STREET, LONDON",183000000.0,51.513980,-0.075425,"(51.5139801, -0.0754254)"
5433,"HANOVER SQUARE, LONDON",165500000.0,51.513908,-0.143824,"(51.51390775, -0.14382390827702274)"
2100,"CARTER LANE, LONDON",159660000.0,51.512973,-0.100203,"(51.5129732, -0.1002027)"
6223,"IMPERIAL ROAD, LONDON",157940000.0,51.602725,-0.122014,"(51.6027248, -0.1220144)"
2879,"COOPERS ROW, LONDON",152706477.6,51.511590,-0.077344,"(51.5115896, -0.077344)"
12407,"WHITEHALL, LONDON",107400000.0,51.502328,-0.126083,"(51.5023278, -0.1260826)"


In [134]:
PriceData_London_ByPriceFixed = PriceData_London_ByPriceFixed.drop(columns='street_coordinates')
PriceData_London_ByPriceFixed

,Street,Avg_Price,Latitude,Longitude
5203,"GRESHAM STREET, LONDON",411500000.0,51.516232,-0.096448
4967,"GODLIMAN STREET, LONDON",302401851.0,51.512526,-0.099089
8986,"PENNANT MEWS, LONDON",194800000.0,51.495338,-0.191601
7845,"MICHAEL ROAD, LONDON",189089600.0,51.566179,0.011433
193,"ALDGATE HIGH STREET, LONDON",183000000.0,51.513980,-0.075425
5433,"HANOVER SQUARE, LONDON",165500000.0,51.513908,-0.143824
2100,"CARTER LANE, LONDON",159660000.0,51.512973,-0.100203
6223,"IMPERIAL ROAD, LONDON",157940000.0,51.602725,-0.122014
2879,"COOPERS ROW, LONDON",152706477.6,51.511590,-0.077344
12407,"WHITEHALL, LONDON",107400000.0,51.502328,-0.126083


In [135]:
#Now we want to create a map of the UK

address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of London are {}, {}.'.format(latitude, longitude))

The coordinates of London are 51.5073219, -0.1276474.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [143]:
# create map of London using latitude and longitude values
LondonMap = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(PriceData_London_ByPriceFixed['Latitude'], PriceData_London_ByPriceFixed['Longitude'], PriceData_London_ByPriceFixed['Avg_Price'], PriceData_London_ByPriceFixed['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(LondonMap)  
    
LondonMap


In [ ]:
#Login to FOURSQUARE

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [171]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [189]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
VenuesPerStreet = getNearbyVenues(names=PriceData_London_ByPriceFixed['Street'],
                                   latitudes=PriceData_London_ByPriceFixed['Latitude'],
                                   longitudes=PriceData_London_ByPriceFixed['Longitude']
                                  )

GRESHAM STREET, LONDON
GODLIMAN STREET, LONDON
PENNANT MEWS, LONDON
MICHAEL ROAD, LONDON
ALDGATE HIGH STREET, LONDON
HANOVER SQUARE, LONDON
CARTER LANE, LONDON
IMPERIAL ROAD, LONDON
COOPERS ROW, LONDON
WHITEHALL, LONDON


In [190]:
VenuesPerStreet

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"GRESHAM STREET, LONDON",51.516232,-0.096448,Pilpel,51.515195,-0.098462,Falafel Restaurant
1,"GRESHAM STREET, LONDON",51.516232,-0.096448,Virgin Active,51.517952,-0.097651,Gym / Fitness Center
2,"GRESHAM STREET, LONDON",51.516232,-0.096448,Museum of London,51.518019,-0.096060,History Museum
3,"GRESHAM STREET, LONDON",51.516232,-0.096448,Postman's Park,51.516860,-0.097643,Park
4,"GRESHAM STREET, LONDON",51.516232,-0.096448,One New Change Rooftop,51.513912,-0.095775,Roof Deck
5,"GRESHAM STREET, LONDON",51.516232,-0.096448,Christ Church Greyfriars Garden,51.515670,-0.098760,Garden
6,"GRESHAM STREET, LONDON",51.516232,-0.096448,M&S Simply Food,51.513590,-0.095297,Grocery Store
7,"GRESHAM STREET, LONDON",51.516232,-0.096448,Manicomio,51.515369,-0.095414,Italian Restaurant
8,"GRESHAM STREET, LONDON",51.516232,-0.096448,Burger & Lobster,51.513687,-0.094643,Seafood Restaurant
9,"GRESHAM STREET, LONDON",51.516232,-0.096448,Daunt Books,51.513982,-0.092995,Bookstore


In [203]:
#We don't care about what type of restaurant we are talking about, so if "restaurant" is in the word, group all as Restaurant
VenuesPerStreet['Venue Category'] = np.where(VenuesPerStreet['Venue Category'].str.contains("Restaurant", case=False, na=False), 'Restaurant', VenuesPerStreet['Venue Category'])
VenuesPerStreet['Venue Category'] = np.where(VenuesPerStreet['Venue Category'].str.contains("Bar", case=False, na=False), 'Bar', VenuesPerStreet['Venue Category'])

VenuesPerStreet

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"GRESHAM STREET, LONDON",51.516232,-0.096448,Pilpel,51.515195,-0.098462,Restaurant
1,"GRESHAM STREET, LONDON",51.516232,-0.096448,Virgin Active,51.517952,-0.097651,Gym / Fitness Center
2,"GRESHAM STREET, LONDON",51.516232,-0.096448,Museum of London,51.518019,-0.096060,History Museum
3,"GRESHAM STREET, LONDON",51.516232,-0.096448,Postman's Park,51.516860,-0.097643,Park
4,"GRESHAM STREET, LONDON",51.516232,-0.096448,One New Change Rooftop,51.513912,-0.095775,Roof Deck
5,"GRESHAM STREET, LONDON",51.516232,-0.096448,Christ Church Greyfriars Garden,51.515670,-0.098760,Garden
6,"GRESHAM STREET, LONDON",51.516232,-0.096448,M&S Simply Food,51.513590,-0.095297,Grocery Store
7,"GRESHAM STREET, LONDON",51.516232,-0.096448,Manicomio,51.515369,-0.095414,Restaurant
8,"GRESHAM STREET, LONDON",51.516232,-0.096448,Burger & Lobster,51.513687,-0.094643,Restaurant
9,"GRESHAM STREET, LONDON",51.516232,-0.096448,Daunt Books,51.513982,-0.092995,Bookstore


In [204]:
VenuesPerStreet.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
"ALDGATE HIGH STREET, LONDON",100,100,100,100,100,100
"CARTER LANE, LONDON",80,80,80,80,80,80
"COOPERS ROW, LONDON",80,80,80,80,80,80
"GODLIMAN STREET, LONDON",73,73,73,73,73,73
"GRESHAM STREET, LONDON",100,100,100,100,100,100
"HANOVER SQUARE, LONDON",100,100,100,100,100,100
"IMPERIAL ROAD, LONDON",8,8,8,8,8,8
"MICHAEL ROAD, LONDON",28,28,28,28,28,28
"PENNANT MEWS, LONDON",87,87,87,87,87,87


In [205]:
#Find out how many unique categories there are
print('There are {} uniques categories.'.format(len(VenuesPerStreet['Venue Category'].unique())))

There are 103 uniques categories.


In [206]:
VenuesPerStreet.shape

(710, 7)

In [207]:
# one hot encoding
venues_onehot = pd.get_dummies(VenuesPerStreet[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = VenuesPerStreet['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Aquarium,Arcade,Art Gallery,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,...,Street Food Gathering,Supermarket,Tailor Shop,Tea Room,Theater,Toy / Game Store,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,"GRESHAM STREET, LONDON",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"GRESHAM STREET, LONDON",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"GRESHAM STREET, LONDON",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"GRESHAM STREET, LONDON",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"GRESHAM STREET, LONDON",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Aquarium,Arcade,Art Gallery,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,...,Street Food Gathering,Supermarket,Tailor Shop,Tea Room,Theater,Toy / Game Store,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,"ALDGATE HIGH STREET, LONDON",0.00,0.000000,0.000000,0.02,0.0100,0.010000,0.110000,0.000000,0.000000,...,0.0100,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.00,0.00
1,"CARTER LANE, LONDON",0.00,0.000000,0.000000,0.00,0.0000,0.025000,0.087500,0.000000,0.000000,...,0.0000,0.000000,0.00,0.012500,0.000000,0.00,0.000000,0.000000,0.00,0.00
2,"COOPERS ROW, LONDON",0.00,0.000000,0.000000,0.00,0.0125,0.012500,0.125000,0.000000,0.000000,...,0.0125,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
3,"GODLIMAN STREET, LONDON",0.00,0.000000,0.000000,0.00,0.0000,0.027397,0.082192,0.013699,0.000000,...,0.0000,0.000000,0.00,0.013699,0.000000,0.00,0.000000,0.000000,0.00,0.00
4,"GRESHAM STREET, LONDON",0.00,0.000000,0.000000,0.03,0.0000,0.020000,0.070000,0.000000,0.000000,...,0.0000,0.000000,0.00,0.010000,0.010000,0.00,0.000000,0.000000,0.01,0.01
5,"HANOVER SQUARE, LONDON",0.01,0.000000,0.000000,0.05,0.0000,0.020000,0.050000,0.000000,0.000000,...,0.0000,0.010000,0.01,0.010000,0.010000,0.02,0.000000,0.000000,0.03,0.00
6,"IMPERIAL ROAD, LONDON",0.00,0.000000,0.000000,0.00,0.0000,0.250000,0.125000,0.000000,0.000000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
7,"MICHAEL ROAD, LONDON",0.00,0.000000,0.000000,0.00,0.0000,0.035714,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.035714,0.035714,0.00,0.00
8,"PENNANT MEWS, LONDON",0.00,0.000000,0.000000,0.00,0.0000,0.000000,0.045977,0.000000,0.022989,...,0.0000,0.011494,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00
9,"WHITEHALL, LONDON",0.00,0.018519,0.018519,0.00,0.0000,0.000000,0.055556,0.000000,0.000000,...,0.0000,0.000000,0.00,0.037037,0.018519,0.00,0.000000,0.000000,0.00,0.00


In [209]:
# What are the top 10 venues/facilities nearby?

num_top_venues = 10

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALDGATE HIGH STREET, LONDON----
                  venue  freq
0            Restaurant  0.31
1                   Bar  0.11
2                 Hotel  0.11
3           Coffee Shop  0.09
4                   Pub  0.04
5  Gym / Fitness Center  0.04
6           Salad Place  0.03
7                  Café  0.03
8           Pizza Place  0.03
9           Art Gallery  0.02


----CARTER LANE, LONDON----
                  venue  freq
0            Restaurant  0.26
1           Coffee Shop  0.10
2                   Pub  0.10
3                   Bar  0.09
4  Gym / Fitness Center  0.05
5        Sandwich Place  0.04
6                 Plaza  0.04
7                Bakery  0.02
8                  Park  0.02
9         Grocery Store  0.02


----COOPERS ROW, LONDON----
                  venue  freq
0            Restaurant  0.22
1                 Hotel  0.14
2                   Bar  0.12
3           Coffee Shop  0.06
4  Gym / Fitness Center  0.05
5                Castle  0.04
6                Garden  0.04
7   

In [210]:
# Define a function to return the most common venues nearby

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [211]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [212]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [214]:
venues_sorted.head(10)

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"ALDGATE HIGH STREET, LONDON",Restaurant,Hotel,Bar,Coffee Shop,Gym / Fitness Center,Pub,Pizza Place,Salad Place,Café,Art Gallery
1,"CARTER LANE, LONDON",Restaurant,Pub,Coffee Shop,Bar,Gym / Fitness Center,Sandwich Place,Plaza,Salad Place,Park,Grocery Store
2,"COOPERS ROW, LONDON",Restaurant,Hotel,Bar,Coffee Shop,Gym / Fitness Center,Garden,Castle,Pub,Salad Place,Scenic Lookout
3,"GODLIMAN STREET, LONDON",Restaurant,Coffee Shop,Pub,Bar,Sandwich Place,Bakery,Salad Place,Pizza Place,Park,Gym / Fitness Center
4,"GRESHAM STREET, LONDON",Restaurant,Coffee Shop,Bar,Gym / Fitness Center,Art Gallery,Plaza,Steakhouse,Scenic Lookout,Café,Burger Joint
5,"HANOVER SQUARE, LONDON",Restaurant,Clothing Store,Art Gallery,Bar,Cosmetics Shop,Boutique,Café,Shoe Store,Lounge,Women's Store
6,"IMPERIAL ROAD, LONDON",Bakery,Pub,Restaurant,Bar,Grocery Store,Breakfast Spot,Café,Yoga Studio,Factory,Deli / Bodega
7,"MICHAEL ROAD, LONDON",Restaurant,Café,Pub,Grocery Store,Bus Stop,Bakery,Music Venue,Burger Joint,Pizza Place,Coffee Shop
8,"PENNANT MEWS, LONDON",Restaurant,Hotel,Pub,Bar,Garden,Café,Historic Site,Grocery Store,Breakfast Spot,Coffee Shop
9,"WHITEHALL, LONDON",Pub,Plaza,Hotel,Café,Bar,Monument / Landmark,Outdoor Sculpture,Historic Site,Coffee Shop,Tea Room


In [215]:
venues_sorted.shape

(10, 11)

In [216]:
london_grouped.shape

(10, 104)

# Results & Discussion

The stakeholders in the company had decided that the best place to open a restaurant would be somewhere where there are as few restaurants as possible already.
Furthermore, it should be one of the most expensive places in London, as the owners want to show the exclusivity of the restaurant and attract customers with above average salaries.

We started by taking the top ten most expensive streets in London and have looked at the venues nearby.

Most of the streets have restaurants nearby, however Whitehall and Imperial Road do not have as many restaurants.

From the above we can conclude, that either Whitehall or Imperial Road would be the optimal areas to open a new restaurant. 


# Conclusion 

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of housing. We recommend venues by finding if there are any nearby restaurants already. 

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend where to start a restaurant.

The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing. 

The managers and stakeholders of EATWELL Corp have looked at the data and decided to start looking at either Whitehall or Imperial Road to find a place to open a restaurant near one of those roads.
In the real world, obviously more factors would come into play when choosing a venue, however with the data available to this research, these two roads have been chosen.
